<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project - TheiaVision: Object Detection Technology for PMD Safety Alerts

> Authors: Ng Wei
---

**Problem Statement:**  
How can we enhance the safety of Personal Mobility Devices (PMDs) in urban environments by using object detection to improve PMD users' ability to perceive and respond to their surroundings?

**Target Audience:**
Management Team of PMD Maker

**Summary:**
This project aims to develop a object detection system to identifies obstacles such as pedestrians, vehicles, and traffic signs. By leveraging CNN algorithms YOLO model, this system would help Darren, a project manager, to lead the development of an alert system with object detection technology

There are a total of three notebooks for this project:  
 1. `01_EDA.ipynb`   
 2. `02_Modelling_Pytorch_SimpleCNN_SGTrafficSign.ipynb`   
 3. `03_Modelling_YOLOv8_labeled_SGTrafficSign.ipynb`
 4. `04_Merge_MultiDataset.ipynb`
 5. `05_Modelling_YOLOv8_combined_data.ipynb`
 6. `06_YOLOv8_Hyperparameter_Tuning.ipynb`

---
**This Notebook**
- We will look at the performace of YOLOv8 with Singapore Traffic Sign dataset.

# 3.1 Importing Libraries
This section of the code is responsible for importing the necessary libraries that will be used in the program.


In [7]:
import os
from IPython import display
display.clear_output()
from IPython.display import display, Image

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Setup complete ✅ (20 CPUs, 15.4 GB RAM, 111.2/1006.9 GB disk)


# 3.2 Download Singapore Traffic Sign Dataset after labeled on Roboflow
This section of the code is loading Roboflow workspace.

In [8]:
#!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Uf5F9mzBsL27ozIYiWvW")
project = rf.workspace("mangguai").project("obstacle-detection-pgo7d")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Obstacle-Detection-3 in yolov8:: 100%|██████████| 5237/5237 [00:00<00:00, 27400.52it/s]


# 3.3 YOLOv8 Pre-trained modelling with labeled Singapore Traffic Sign Dataset

In [9]:
import os

# Define the classes and their corresponding indices
class_names = ['nojaywalking_sign','mandatorystop_sign','curveright_sign','zebracrossing_sign','noentry_sign','pedestriancrossing_sign','curveleft_sign']
class_indices = {name: index for index, name in enumerate(class_names)}

# Image dimensions
img_width, img_height = 640, 640

# Path to the folder containing the annotation files
folder_path = 'Obstacle-Detection-3/train/labels'
output_folder_path = 'Obstacle-Detection-3/train/modified_labels'  # New output folder

os.makedirs(output_folder_path, exist_ok=True)  # Ensure the output directory exists

# Function to normalize coordinates
def normalize_coordinates(coord, max_value):
    return float(coord) / max_value

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # assuming the annotation files are .txt files
        file_path = os.path.join(folder_path, filename)
        output_file_path = os.path.join(output_folder_path, filename)  # Output file path
        with open(file_path, 'r') as file:
            lines = file.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split(' ')
            if len(parts) == 10:
                # Extract label and coordinates
                label = parts[-2]
                coords = parts[:8]

                # Get the class index
                class_index = class_indices.get(label, -1)
                if class_index != -1:
                    # Normalize coordinates
                    normalized_coords = [normalize_coordinates(coord, img_width if i % 2 == 0 else img_height) for i in range(8)]

                    # Convert to the desired format and add to new lines
                    new_line = f"{class_index} " + " ".join(map(str, normalized_coords))
                    new_lines.append(new_line)

        # Write the converted lines to a new file in the output directory
        with open(output_file_path, 'w') as file:
            file.write('\n'.join(new_lines))


In Python, command could be a variable that holds some value, or it could be a function that performs a certain operation when called. However, without more information, it's impossible to say what command does in your code.

If command is a function, it would be called like this: command(). If it's a variable, it might be used in an expression like print(command) to display its value, or x = command to assign its value to another variable.

Please provide more context or the complete line of code where command is used for a more accurate explanation.

In [14]:
import subprocess

# Command to run
command = "yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Obstacle-Detection-3/data.yaml epochs=20 imgsz=640 batch=8"

# Execute the command
subprocess.run(command, shell=True) 

New https://pypi.org/project/ultralytics/8.2.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/mangguai/capstone/code/Obstacle-Detection-3/data.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, lin

CompletedProcess(args='yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Obstacle-Detection-3/data.yaml epochs=20 imgsz=640 batch=8', returncode=0)

- The YOLOv8 model was trained using Ultralytics YOLOv8.0.196 on a Python 3.11.8 and PyTorch 2.2.2 environment, leveraging a CUDA-enabled NVIDIA GeForce RTX 3060 GPU. The training involved adjusting the model to 7 classes, processing 2360 images over 20 epochs, and validating on 169 images. Significant performance gains were noted, with final metrics showing an mAP50 of 0.961 and an mAP50-95 of 0.914, indicating highly accurate detection capabilities. The training, enhanced by Automatic Mixed Precision, was efficient, completing in approximately 0.306 hours.